# Quantum Numbers

## Overview

In this notebook we will introduce the idea of quantum number conservation and how to enforce it in Renormalizer.

## Quantum number conservation

In the context of Shuai group's research, the most common case of quantum number conservation is the conservation of electron/exciton in the Holstein model.

$$
\hat H = \sum_{ij} J_{ij} \hat a^\dagger_i \hat a_{i+1} + \sum_{ik}\frac{1}{2}  (\hat p_{ik}^2 + \omega_k^2 \hat x_{ik}^2) + \sum_{ik} \hat a^\dagger_i \hat a_i \hat x_{ik}
$$

Here, each electronic degree of freedom, described by $\hat a^\dagger_i$ and $\hat a$, is coupled with a set of harmonic oscillators indexed by $k$.

For the demonstration purposes, we may neglect the vibrations and focus only on the electronic part, which leads us to the Hückel model (or tight binding model)

$$
\hat H = \sum_{ij} J_{ij} \hat a^\dagger_i \hat a_{i+1}
$$

This is a model that conserves the total particle number. Formally speaking, this means $\hat H$ commutes with the total particle number operator $\hat N = \sum_i \hat a^\dagger_i \hat a_i$.
More intuitively, for any state $|\psi\rangle$ with a particular particle number, $\hat H |\psi\rangle$ has the same particle number.
For example, suppose there are only two sites, and 
$$
|\psi\rangle = |01\rangle+|10\rangle
$$

Thus,

$$
\hat H |\psi\rangle = J_{12}|10\rangle+J_{21}|01\rangle
$$

The two states with different quantum numbers, $|00\rangle$ and $|11\rangle$, will not present in $\hat H |\psi\rangle$.
In this model, the total particle number, is called a (good) quantum number.

There are two main reasons why we should care about quantum number
- For many chemical applications the quantum number can be considered as an input to the model, just as the coefficients $J_{ij}$. For example, the total number of exciton in a excitonic coupling model should be 1. If the conservation of quantum number is not enforced, numerical error may eventually lead us to incorrect solution.
- Another advantage of using quantum number is that it saves memory and accelerates calculation. By looking at quantum numbers we can assert some of the matrix/vector elements must be zero. The sparsity can then be exploited for more efficient tensor manipulation, such as SVD.

## Setting quantum number for states

In renormalizer, the first place to set the quantum number is basis sets. Most `BasisSet` classes have the `sigmaqn` argument that determines the quantum number of each basis.

In [7]:
from renormalizer import BasisSimpleElectron

Here we set up two basis sets and sets the quantum number for $|0\rangle$ to 0 and $|1\rangle$ to 1.

In [8]:
b1 = BasisSimpleElectron(0, sigmaqn=[0, 1])
b2 = BasisSimpleElectron(1, sigmaqn=[0, 1])
basis = [b1, b2]

We next build a random MPS based on the basis sets and see its effect

In [9]:
from renormalizer import Mps, Model

In [10]:
model = Model(basis, ham_terms=[])

In [11]:
mps = Mps.random(model, qntot=1, m_max=2)

In [12]:
mps[0].array

array([[[1., 0.],
        [0., 1.]]])

In [13]:
mps[1].array

array([[[ 0.        ],
        [-0.7089592 ]],

       [[-0.70524949],
        [ 0.        ]]])

In [24]:
mps.todense()

array([ 0.        , -0.7089592 , -0.70524949,  0.        ])

We can see that although the MPS is random, half of the matrix elements are zero, due to particle number conservation. As a result, the overall dense state vector has a well-defined particle number of 1.

For comparison, in the following the MPS when quantum number is not activated is shown

In [19]:
model2 = Model([BasisSimpleElectron(i, sigmaqn=[0, 0]) for i in range(2)], ham_terms=[])
mps2 = Mps.random(model2, qntot=0, m_max=2)

In [20]:
mps2[0].array

array([[[-0.77598623,  0.63074984],
        [-0.63074984, -0.77598623]]])

In [21]:
mps2[1].array

array([[[ 0.70367869],
        [ 0.69899228]],

       [[-0.1140342 ],
        [-0.05694114]]])

In [26]:
mps2.todense()

array([-0.61797203, -0.578324  , -0.35535626, -0.39670373])

## Setting quantum number for operators
Just as states, operators are also associated with quantum number. The quantum number of an operator shows the change of quantum number if the operator is applied to a state.
For example, the quantum number of $\hat a^\dagger$ is 1, since

$$
\hat a^\dagger |0\rangle = |1\rangle
$$

Similarly, the quantum number of $\hat a$ is -1.

In Renormalizer, to fully take advantage of quatnum number conservation, we need to set the quantum number for the operators. The `Op` class accepts the `qn` argument for the quantum number of each elementary operator.

In [31]:
from renormalizer import Op, Mpo

Apply the creation operator to the MPS. The total quantum number of the MPS increases from 1 to 2, and the resulting state is $|11\rangle$.

In [34]:
mps3 = Mpo(model, Op(r"a^\dagger", 0, qn=1)) @ mps

2024-07-26 11:13:26,200[DEBUG] # of operator terms: 1


In [38]:
mps3.qntot

array([2])

In [39]:
mps3.todense()

array([ 0.       ,  0.       ,  0.       , -0.7089592])

For complex symbols the quantum number for each elementary symbol should be specified.

In [29]:
ham_terms = Op(r"a^\dagger a", [0, 1], qn=[1, -1]) + Op(r"a^\dagger a", [1, 0], qn=[1, -1])

To summarize, three places are relevant to the setup of quantum number
- The basis sets
- The operators
- Total quantum number in MPS

## Advanced usage (To be updated)

- Setting quantum number is not necessary. When quantum number is not conserved or in other cases when quantum number is not desired, we can skip this by set all to 0.

- If quantum numbers are set incorrectly.

- U(1) quantum number and more than one quantum number